In [ ]:
import pandas as pd
import psycopg2
import pyodbc
from datetime import datetime, date
import warnings
from tqdm.notebook import tqdm

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
#Passwords
ONEVIEW_PASSWORD = ''
EDW_USERNAME = '
EDW_PASSWORD = ''
AIMS_USERNAME = ''
AIMS_PASSWORD = ''

In [ ]:
#OneView connection
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password=ONEVIEW_PASSWORD)

In [ ]:
#EDW connection
w = "DSN=prddw; UID={}; PWD={}".format(EDW_USERNAME, EDW_PASSWORD)
AMAEDW = pyodbc.connect(w)

In [ ]:
#Define universe
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [ ]:
#Get party IDs and MEs
ME_QUERY = \
    f"""
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND='Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [ ]:
#Get present employment
MED_QUERY = \
    """
    SELECT
    TOP_ID,
    EMPLOYER_ID,
    PARTY_ID
    FROM
    AMAEDW.MED_PROF
    WHERE
    THRU_DT IS NULL
    """
MED_PROF = pd.read_sql(con=AMAEDW, sql=MED_QUERY)

In [ ]:
#Get mpa
MPA_QUERY = \
    """
    SELECT
    MPA_CD,
    TOP_ID,
    EMPLOYER_ID,
    DESC
    FROM
    AMAEDW.MPA
    WHERE
    ACTIVE_IND='Y'
    """
MPA = pd.read_sql(con=AMAEDW, sql=MPA_QUERY)

In [ ]:
#Get specialty
SPEC_QUERY = \
    """
    SELECT
    M.PARTY_ID,
    M.SPEC_ID,
    M.PREFE_LVL
    FROM
    AMAEDW.MED_SPEC M
    """
SPEC = pd.read_sql(con=AMAEDW, sql=SPEC_QUERY)

In [ ]:
#Get awards
AWARD_QUERY = \
    """
    SELECT
    PARTY_ID,
    EXPIRATION_DT
    FROM
    AMAEDW.PRA_CERTIFICATE
    WHERE
    PRA_STATUS_DESC='Approved'
    """
AWARD = pd.read_sql(con=AMAEDW, sql=AWARD_QUERY)

In [ ]:
#Merge with pandas
AWARD.EXPIRATION_DT = pd.to_datetime(AWARD.EXPIRATION_DT)
wards = AWARD[AWARD.EXPIRATION_DT>datetime.today()].sort_values('EXPIRATION_DT').drop_duplicates('PARTY_ID', keep='last')
ov_universe = pd.merge(party_ids, ov_me, left_on='ME', right_on='medical_education_number')
ALL = pd.merge(ov_universe, MED_PROF, on='PARTY_ID', how='left')
ALL = pd.merge(ALL, SPEC[SPEC.PREFE_LVL==1], on='PARTY_ID', how='left')
ALL = pd.merge(ALL, SPEC[SPEC.PREFE_LVL==2], on='PARTY_ID', suffixes = ['_PRIM','_SEC'], how='left')
ALL = pd.merge(ALL, MPA, on=['TOP_ID','EMPLOYER_ID'], how='left')
ALL = pd.merge(ALL, wards, on='PARTY_ID', how='left')